In [1]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.formula.api as sm
from statsmodels import regression
import statsmodels

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc, rcParams